# Starter notebook
Copy (fork) and edit as many copies of this notebook as you require 

In [21]:
# venv\Scripts\activate

In [22]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 72)

seed = 42
import numpy as np
np.random.seed(seed)

# graphics
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})
plt.rcParams["figure.figsize"] = (11, 6.8)
#plt.style.use('fivethirtyeight')

import seaborn as sns
sns.set(font_scale=1)
#sns.set_style("whitegrid")

import plotly.io as pio
# for use in JupyterLab 4
pio.renderers.default = 'iframe'
# for use in Google Colab
#pio.renderers.default = 'colab'
import plotly as py
import plotly.express as px

## Read in the training data

In [23]:
train = pd.read_csv("train.csv")
train

,id,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,FAGECOMB,FEDUC,ILLB_R,ILOP_R,ILP_R,LD_INDL,MAGER,MBSTATE_REC,MEDUC,M_Ht_In,NO_INFEC,NO_MMORB,NO_RISKS,PAY,PAY_REC,PRECARE,PREVIS,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN,DBWT
0,0,1,1,21.3,0,7,1,5,1319.0,3,28,6,888,888,888,N,25,1,6,68,1,1,1,2,2,2,13,0,0,0,140.0,1,2,N,0,F,37,3572
1,1,1,1,27.1,0,12,2,9,712.0,3,99,9,40,999,999,Y,26,1,3,67,1,1,0,1,1,2,13,0,2,3,173.0,1,1,N,0,F,35,3315
2,2,1,1,27.8,0,4,1,12,853.0,5,25,4,29,888,29,N,25,1,4,63,1,1,0,2,2,4,8,0,2,0,157.0,3,1,N,0,M,41,1910
3,3,1,1,22.3,0,12,1,9,2054.0,2,38,6,888,22,22,N,38,1,6,63,1,1,1,2,2,2,13,0,0,1,126.0,1,1,N,0,F,31,3005
4,4,1,1,22.7,0,2,1,11,944.0,3,29,4,999,999,999,N,30,1,4,64,1,1,1,2,2,2,12,0,2,1,NaN,1,2,N,0,F,18,3714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,6995,1,1,20.2,0,11,1,8,1251.0,5,30,6,33,44,33,N,28,1,6,66,1,1,1,2,2,2,9,0,1,1,125.0,1,2,N,0,M,20,3374
6996,6996,1,1,36.9,0,8,1,5,1757.0,4,37,5,888,36,36,N,31,1,6,64,1,1,1,2,2,2,15,0,0,1,215.0,3,2,N,0,F,29,4195
6997,6997,3,1,44.6,0,9,,6,1746.0,7,20,4,888,888,888,N,18,1,3,64,1,1,1,2,2,2,9,0,0,0,260.0,1,1,N,0,F,0,2900
6998,6998,1,1,24.4,0,1,1,10,NaN,3,33,6,15,888,15,N,32,1,6,70,1,1,0,2,2,2,17,0,1,0,170.0,4,3,Y,1,F,45,3810


## This is the test data that you are asked to make predictions for

In [24]:
test = pd.read_csv("test.csv")
test

,id,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,FAGECOMB,FEDUC,ILLB_R,ILOP_R,ILP_R,LD_INDL,MAGER,MBSTATE_REC,MEDUC,M_Ht_In,NO_INFEC,NO_MMORB,NO_RISKS,PAY,PAY_REC,PRECARE,PREVIS,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,7000,1,1,35.7,0,12,2,9,412.0,3,38,4,29,888,29,Y,34,1,6,71,1,1,1,2,2,3,11,0,1,0,256.0,1,2,N,0,M,24
1,7001,1,1,24.2,0,11,,8,1007.0,7,24,3,888,888,888,N,22,1,3,64,1,1,1,2,2,2,11,0,0,0,141.0,1,1,N,0,F,11
2,7002,1,1,20.2,0,5,2,2,1328.0,5,31,5,888,888,888,N,26,2,7,69,1,1,1,1,1,2,16,0,0,0,137.0,3,2,N,0,M,39
3,7003,1,1,23.0,0,8,,5,2012.0,2,22,4,888,888,888,N,19,1,5,63,1,1,1,1,1,2,10,0,0,0,130.0,1,1,N,0,M,43
4,7004,1,1,37.1,0,3,2,12,NaN,2,33,4,888,999,999,N,33,1,6,67,1,1,0,1,1,3,13,0,0,2,237.0,3,1,N,0,F,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,8995,2,1,20.4,0,11,1,8,1653.0,3,29,7,16,888,16,N,32,1,4,67,1,1,1,2,2,3,7,0,4,0,130.0,1,3,N,0,F,18
1996,8996,1,1,26.2,0,7,1,4,1021.0,6,29,4,888,999,999,Y,27,1,6,67,1,1,1,1,1,2,16,0,0,1,167.0,1,1,N,0,M,54
1997,8997,1,1,35.9,0,9,1,6,1014.0,1,56,6,54,888,54,N,44,2,6,68,1,1,0,2,2,2,11,0,1,0,236.0,4,1,Y,1,F,36
1998,8998,1,1,19.0,0,12,2,9,636.0,2,29,6,74,888,74,N,28,1,4,66,1,1,1,1,1,2,12,0,1,0,118.0,1,1,N,0,F,41


In [25]:
# Structural cleaning before splitting
def structural_cleaning(df, is_train=True):
   df = df.copy()

   # A. Filter target (only train)
   if is_train and 'DBWT' in df.columns:
       df = df[df['DBWT'] != 9999]

   # B. Null codes
   # Map 99, 999 ...
   cols_99 = ['CIG_0', 'FAGECOMB', 'M_Ht_In', 'PREVIS']
   for col in cols_99:
       if col in df.columns:
           df[col] = df[col].replace(99, np.nan)

   cols_9 = ['FEDUC', 'MEDUC', 'BFACIL', 'ATTEND']
   for col in cols_9:
        if col in df.columns:
            df[col] = df[col].replace(9, np.nan)

   if 'BMI' in df.columns:
        df['BMI'] = df['BMI'].replace(99.9, np.nan)

   # C. Structural nullity (888 -> 0)
   cols_888 = ['ILLB_R', 'ILOP_R', 'ILP_R']
   for col in cols_888:
        if col in df.columns:
            df[col] = df[col].replace(888, 0)
            df[col] = df[col].replace(999, np.nan)

   return df

In [26]:
# Apply structural cleaning to every df
train_cleaned = structural_cleaning(train, is_train=True)
test_cleaned = structural_cleaning(test, is_train=False)

Splitting is based into 3 conjuntos
1. Conjunto train: Is used so the model can learn parameters
2. Conjunto validation: To select the best model and set hyperparameters to estimate errors of generalization during development.
3. Conjunto test: Untouchable until the end. Gives a non-biassed estimation about how the model works irl.

In [27]:
# The target variable is 'Delivery Birth Weight' (DBWT)
from sklearn.model_selection import train_test_split

# Separate target variable to isolate the features (X)
y = train_cleaned.pop('DBWT')
X = train_cleaned

# 80/20 ratio
# Split the 20% of data for final testing
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# From the 80% training data, split again into training and validation sets (75/25 ratio)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

print(f"Train shape: {X_train.shape} (aprox 60%)")
print(f"Val shape:   {X_val.shape}   (aprox 20%)")
print(f"Test shape:  {X_test.shape}  (aprox 20%)")

Train shape: (4200, 37) (aprox 60%)
Val shape:   (1400, 37)   (aprox 20%)
Test shape:  (1400, 37)  (aprox 20%)


Implementing a baseline signifies that every model turned in after this must best this result no matter what.

In [28]:
from sklearn.metrics import root_mean_squared_error

# Define the prediction (mean of training target)
baseline_pred = y_train.mean()
print(f"Baseline value: {baseline_pred:.2f}")

# Create a dummy prediction vector array for validation
y_val_pred = np.full(len(y_val), baseline_pred)

# Evaluate RMSE
rmse = root_mean_squared_error(y_val, y_val_pred)
print(f"Baseline RMSE on validation set: {rmse:.2f}")

Baseline value: 3259.56
Baseline RMSE on validation set: 581.88


Statistical imputation is the process of replacing missing data points in a dataset with plausible substituted values to maintaint data integrity. I did the splitting before filling in data to avoid data leakage.

Extracting all the info from `UserGuide2018-508` shared in the Kaggle description I made a full data cleanup to try a standard scaler.

In [29]:
# Columns that do not give predictive value
drop_cols = ['id', 'DLMP_MM', 'DOB_TT'] # Exact dates that give noise

# Numerical columns -> Candidates for median + scaling
# Education has order: We try as a numerical
num_cols = [
    'BMI', 'CIG_0', 'FAGECOMB', 'ILLB_R', 'ILOP_R', 'ILP_R', 'MAGER',
    'M_Ht_In', 'PRECARE', 'PREVIS', 'PRIORDEAD', 'PRIORLIVE', 'PRIORTERM',
    'PWgt_R', 'RF_CESARN', 'WTGAIN', 'FEDUC', 'MEDUC'
]

# Categorical columns -> Candidates for mode + one-hot encoding
# Text variables that do not have order
cat_cols = [
    'ATTEND', 'BFACIL', 'DMAR', 'DOB_MM', 'DOB_WK', 'LD_INDL',
    'MBSTATE_REC', 'PAY', 'PAY_REC', 'RDMETH_REC', 'RESTATUS',
    'RF_CESAR', 'SEX', 'NO_INFEC', 'NO_MMORB', 'NO_RISKS'
]

In [30]:
# Training stats learning
def learn_imputation_stats(df):
   stats = {}

   # A. Numerical -> Median
   for col in num_cols:
      if col in df.columns:
         stats[col] = df[col].median()

   # B. Categorical -> Mode
   for col in cat_cols:
      if col in df.columns:
         # dropna=True to avoid NaN in mode calculation
         stats[col] = df[col].mode(dropna=True)[0]

   return stats
imputation_stats = learn_imputation_stats(X_train)

B. Really important for FAGECOMB -> Not having a father is predictive

In [31]:
from sklearn.preprocessing import StandardScaler

def process_dataset(df, stats, scaler=None, is_train=False):
   df = df.copy()

   # A. Drop useless columns
   df = df.drop([c for c in drop_cols if c in df.columns], axis=1)

   # B. Missing indicators
   cols_with_heavy_nan = ['FAGECOMB']
   for col in cols_with_heavy_nan:
      if col in df.columns:
         # Creates a binary column: 1 if missing, 0 if not
         df[f'{col}_is_missing'] = df[col].isna().astype(int)

   # C. Imputation (fill in the blanks)
   for col, value in stats.items():
      if col in df.columns:
         df[col] = df[col].fillna(value)

   # Security check: if any NaN remains
   df = df.fillna(0)

   # D. One-hot encoding for categorical variables to dummy
   # Change to string first to secure numerical codes are treated as categories
   current_cat_cols = [c for c in cat_cols if c in df.columns]
   for col in current_cat_cols:
      df[col] = df[col].astype(str)

   df = pd.get_dummies(df, columns=current_cat_cols, drop_first=True) # drop_first avoids colineality

   # E. Scaling (StandardScaler)
   current_num_cols = [c for c in num_cols if c in df.columns]

   if is_train:
      scaler = StandardScaler()
      df[current_num_cols] = scaler.fit_transform(df[current_num_cols])
      return df, scaler
   else:
      # if Val/Test we use the scaler already trained (NOT RE-FIT)
      if scaler is not None:
         df[current_num_cols] = scaler.transform(df[current_num_cols])
      return df


Pipeline execution
1. Process TRAIN & obtain adjusted scaler
2. Process VALIDATION using stats & Train scaler
3. Process TEST
4. PROCESS TEST for Submission

In [32]:
# 1
X_train_ready, scaler_fitted = process_dataset(X_train, imputation_stats, is_train=True)
# 2
X_val_ready = process_dataset(X_val, imputation_stats, scaler=scaler_fitted, is_train=False)
# 3
X_test_ready = process_dataset(X_test, imputation_stats, scaler=scaler_fitted, is_train=False)

One-Hot Encoding in the **D** step can generate distinct cols if in test missis some weird category. I force everyone to have EXACTLY the cols of X_train_ready
expected_cols = X_train_ready.columns

In [33]:
expected_cols = X_train_ready.columns

def align_columns(df, target_cols):
    # 1. Add missing cols (rellenas con 0)
    missing_cols = set(target_cols) - set(df.columns)
    for c in missing_cols:
        df[c] = 0

    # 2. Remove extra columns (that weren't in train)
    # 3. Reordenar para que coincidan índice a índice
    return df[target_cols]

X_val_ready = align_columns(X_val_ready, expected_cols)
X_test_ready = align_columns(X_test_ready, expected_cols)

print(f"Train final shape: {X_train_ready.shape}")
print(f"Val final shape:   {X_val_ready.shape}")
print(f"Test final shape:  {X_test_ready.shape}")

Train final shape: (4200, 74)
Val final shape:   (1400, 74)
Test final shape:  (1400, 74)


In [ ]:
from sklearn.linear_model import LinearRegression

# Entrenar
model = LinearRegression()
model.fit(X_train_ready, y_train)

# Predecir y evaluar
preds = model.predict(X_val_ready)
rmse = root_mean_squared_error(y_val, preds)

print(f"Linear Regression RMSE: {rmse:.2f}")

Linear Regression RMSE: 550.86


In [37]:
# Process the true kaggle test
test_kaggle_ready = process_dataset(test_cleaned, imputation_stats, scaler=scaler_fitted, is_train=False)
test_kaggle_ready = align_columns(test_kaggle_ready, expected_cols)

print(f"Test Kaggle shape: {test_kaggle_ready.shape}") # Debería ser (2000, 74)

kaggle_predictions = model.predict(test_kaggle_ready)

Test Kaggle shape: (2000, 74)


## Submit your predictions in a `submission.csv` file for scoring on the [leaderboard](https://www.kaggle.com/competitions/u-tad-birth-weight-point-prediction-2025/leaderboard)
To submit your notebook click on **Submit to competition** and then **Submit**.

In [38]:
# do not modify this code
submission = pd.read_csv("sample_submission.csv")
submission["DBWT"] = kaggle_predictions
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.head()